In [ ]:
from pathlib import Path
import os 

import matplotlib.pyplot as plt
import numpy as np
import auxiliary_functions as aux

import sirf.Gadgetron as pMR

# this is where we store the properly formatted data
root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"
fpath_input = root_path / "Input"
fpath_output = root_path / "Output"

fname_stat          = fpath_output / "output_d_simulate_mrf_static_1500.h5"
fname_resp          = fpath_output / "output_g_simulate_mrf_resp.h5"
fname_cardioresp    = fpath_output / "output_g_simulate_mrf_cardio_resp.h5"

In [ ]:
ad_stat = pMR.AcquisitionData(str(fname_stat))
ad_resp = pMR.AcquisitionData(str(fname_resp))
ad_cardioresp = pMR.AcquisitionData(str(fname_cardioresp))

num_recon_imgs = 3
num_cg_iter = 5

# 
fname_out = fpath_output / "output_h_acquisitions_resolved_stat.h5"

if fname_out.is_file():
    ad_stat = pMR.AcquisitionData(str(fname_out))
    recon_stat = pMR.ImageData()
    recon_stat.from_acquisition_data(ad_stat)
    
    fname_in = fpath_output / "output_h_recon_resolved_stat.npy"
    arr = np.load(str(fname_in))
    recon_stat.fill(arr)

else:
    recon_stat, ad_stat  = aux.iterative_reconstruct_timeresolved(ad_stat,num_recon_imgs,num_cg_iter)
    ad_stat.write(str(fname_out))

    fname_out = fpath_output / "output_h_recon_resolved_stat.npy"
    np.save(str(fname_out), recon_stat.as_array())

In [ ]:
# split cells to enable sequential execution

fname_out = fpath_output / "output_h_acquisitions_resolved_resp.h5"

if fname_out.is_file():
    ad_resp = pMR.AcquisitionData(str(fname_out))
    recon_resp = pMR.ImageData()
    recon_resp.from_acquisition_data(ad_resp)
    
    fname_in = fpath_output / "output_h_recon_resolved_stat.npy"
    arr = np.load(str(fname_in))
    recon_resp.fill(arr)

else:
    recon_resp, ad_resp  = aux.iterative_reconstruct_timeresolved(ad_resp,num_recon_imgs,num_cg_iter)
    ad_resp.write(str(fname_out))

    fname_out = fpath_output / "output_h_recon_resolved_resp.npy"
    np.save(str(fname_out), recon_resp.as_array())

In [ ]:
# 
fname_out = fpath_output / "output_h_acquisitions_resolved_cardioresp.h5"

if fname_out.is_file():
    ad_cardioresp = pMR.AcquisitionData(str(fname_out))
    recon_cardioresp = pMR.ImageData()
    recon_cardioresp.from_acquisition_data(ad_resp)
    
    fname_in = fpath_output / "output_h_recon_resolved_cardioresp.npy"
    arr = np.load(str(fname_in))
    recon_cardioresp.fill(arr)

else:
    recon_cardioresp, ad_cardioresp = aux.iterative_reconstruct_timeresolved(ad_cardioresp,num_recon_imgs,num_cg_iter)
    ad_cardioresp.write(str(fname_out))

    fname_out = fpath_output / "output_h_recon_resolved_cardioresp.npy"
    np.save(str(fname_out), recon_cardioresp.as_array())

In [ ]:
# finally perform dictionary matching
fname_dict = Path("/media/sf_CCPPETMR/TestData/Input/xDynamicSimulation/pDynamicSimulation/Fingerprints/dict_70_1500.npz")
mrfdict = np.load(fname_dict)

dict_theta = mrfdict['dict_theta']
dict_mrf = mrfdict['dict_norm']

dict_stat = np.transpose( aux.apply_databased_sliding_window(ad_stat, np.transpose(dict_mrf)))
dict_resp = np.transpose( aux.apply_databased_sliding_window(ad_resp, np.transpose(dict_mrf)))
dict_cardioresp = np.transpose( aux.apply_databased_sliding_window(ad_cardioresp, np.transpose(dict_mrf)))

In [ ]:
# split cells to enable sequential execution
match_stat = aux.dictionary_matching(recon_stat, dict_stat, dict_theta)

In [ ]:
match_resp = aux.dictionary_matching(recon_resp, dict_resp, dict_theta)

In [ ]:
match_cardioresp = aux.dictionary_matching(recon_cardioresp, dict_cardioresp, dict_theta)

In [ ]:
import nibabel as nib
T1GT = nib.load(str(fpath_output/ "output_c_static_ground_truth_T1_ms.nii"))
T2GT = nib.load(str(fpath_output/ "output_c_static_ground_truth_T2_ms.nii"))

f,ax = plt.subplots(2,4)

ax[0,0].imshow(np.abs(match_stat[:,:,1]),cmap='jet',vmin=400,vmax=2500)
ax[0,0].axis('off')
ax[0,2].set_title('S')
ax[1,0].imshow(np.abs(match_stat[:,:,2]),cmap='viridis',vmin=10,vmax=150)
ax[1,0].axis('off')

ax[0,1].imshow(np.abs(match_resp[:,:,1]),cmap='jet',vmin=400,vmax=2500)
ax[0,1].axis('off')
ax[0,2].set_title('R')
ax[1,1].imshow(np.abs(match_resp[:,:,2]),cmap='viridis',vmin=10,vmax=150)
ax[1,1].axis('off')

ax[0,2].imshow(np.abs(match_cardioresp[:,:,1]),cmap='jet',vmin=400,vmax=2500)
ax[0,2].axis('off')
ax[0,2].set_title('C+R')
ax[1,2].imshow(np.abs(match_cardioresp[:,:,2]),cmap='viridis',vmin=10,vmax=150)
ax[1,2].axis('off')

ax[0,3].imshow(np.transpose(np.abs(np.squeeze(T1GT.get_fdata()))),cmap='jet',vmin=400,vmax=2500)
ax[0,3].axis('off')
ax[0,3].set_title('GT')
ax[1,3].imshow(np.transpose(np.abs(np.squeeze(T2GT.get_fdata()))),cmap='viridis',vmin=10,vmax=150)
ax[1,3].axis('off')

fig_path = root_path / "Figures"
fig_path.mkdir(exist_ok=True)

fname_out = fig_path / "fig_h_dictionary_matching_motionresolved.png"
plt.savefig(str(fname_out), dpi=300)
plt.show()